# 🧠 Project 01 — Artificial Intelligence 2025/26  
## Class Timetable (CSP Problem)

---

### 1. Introduction
> This project aims to develop an intelligent agent capable of generating class timetables for the undergraduate courses at the Polytechnic Institute of Cávado and Ave (IPCA).  
> Creating timetables is a complex and time-consuming task that involves multiple constraints related to professors, subjects, classrooms, and student groups. Traditionally, this process requires significant time and effort from the administrative team.  
> In this context, the project intends to apply Artificial Intelligence techniques, specifically Constraint Satisfaction Problems (CSP), to automate the creation of viable and optimized timetables, respecting the imposed conditions and minimizing conflicts.  
> The project will be implemented in Python, using the `python-constraint` library, and fully documented in a Jupyter Notebook, as per the guidelines of the Artificial Intelligence (AI) course.

**Group members:**
- Pedro Ribeiro — student number 27960  
- Ricardo Fernandes — student number 27961  
- Carolina Branco — student number 27983  
- João Barbosa — student number 27964  
- Diogo Abreu — student number 27975  

---

### 2. Goal Formulation

The main goal of this project is to **design an intelligent agent** capable of automatically generating valid class timetables for undergraduate courses at IPCA.  

The agent must assign **courses to time slots and rooms**, ensuring that no scheduling conflicts occur and that all academic and logistical constraints are satisfied.  

#### Limitations and Constraints
The timetable generation process is subject to several limitations, including:

- **Teacher availability** — each lecturer may have unavailable time slots.  
- **Room capacity and restrictions** — some courses must occur in specific rooms (e.g., labs).  
- **Course assignment** — each class is associated with a specific set of courses.  
- **Online lessons** — some sessions may occur online and not require a physical room.  
- **Non-overlapping sessions** — a class, teacher, or room cannot be used in two sessions simultaneously.  
- **Class duration** — each lesson lasts 2 hours.  
- **Weekly lessons per class** — all classes have 10 lessons per week.  
- **Lessons per course** — each course may have 1 or 2 lessons per week.  
- **Daily lesson limit** — a class cannot have more than 3 lessons per day.  

#### Expected Results
The system should produce a **valid timetable solution** that:
- Assigns every course session to a valid time slot and room.  
- Respects all **hard constraints**.  
- Minimizes conflicts and overlaps between classes and teachers.  
- Optionally considers **soft constraints** to optimize the timetable.

---

### 3. Problem Formulation (CSP)

The timetable problem was modeled as a **Constraint Satisfaction Problem (CSP)** using the `python-constraint` library.

#### Variables
Each **lesson** of a course is represented by two variables:
- `<course>_<lesson>_slot` — the time slot assigned to that lesson.  
- `<course>_<lesson>_room` — the classroom assigned to that lesson.

#### Domains
- **Time slots:** `1–20` (representing possible periods in the week).  
- **Rooms:** `{Lab01, Room1, Room2}`, unless specific restrictions apply.  

#### Constraints

**Hard Constraints (mandatory):**
- Classes last 2 hours.  
- All classes have 10 lessons per week.  
- Each course may have 1 or 2 lessons per week.  
- A class cannot have more than 3 lessons per day.  
- Teacher availability — a course cannot be scheduled in time slots where its lecturer is unavailable.  
- Room restrictions — some courses are restricted to specific rooms (e.g., labs).  
- No overlapping classes — no class, teacher, or room can appear in two lessons at the same time.  

**Soft Constraints (preferred, can be violated if necessary):**
- Lessons of the same course must be scheduled on distinct days.  
- Each class should have only four days of lessons per week.  
- Lessons within the same day should be consecutive. 

#### Additional Constraints (optional improvements)

**Hard Constraints (additional):**
- When online classes are scheduled, limited to a maximum of three per course, they must be scheduled on the same day.  
- Some classes are required to be assigned to a specific classroom (already partly implemented via `roomrestrictions`).  

**Soft Constraints (additional):**
- The number of classrooms used by each class should be minimized.  

#### Heuristics
The CSP solver may apply **variable ordering heuristics** (e.g., most constrained variable first) and **domain reduction** to improve efficiency.  
Further heuristics (e.g., least constraining value) may be explored in later iterations.

---

### 5. Repository Link
> 🔗 **GitHub Repository:** [https://github.com/diogooaabreu/IA25_P01_G4.git](https://github.com/diogooaabreu/IA25_P01_G4.git)

---

### 4. Implementation

> The implementation is based on the `python-constraint` library.  
> The dataset is read from `datasets/timetable_dataset.txt`, and constraints are dynamically built from the data.  
> Below is the initial implementation of the CSP model:

---




In [6]:
import json
from constraint import *
from collections import Counter 
import itertools
import time


# Definição das constantes e funções de restrição (baseado no seu código)
# =========================================================================

rooms = ["Lab01", "Room1", "Room2", "Online"]
lessons_per_course = 1
slots = list(range(1, 21))

def no_consecutive_slots(*slots):
    slots = sorted([s for s in slots if s is not None])
    for i in range(len(slots) - 1):
        if slots[i + 1] - slots[i] == 1:
            return False
    return True

def room_no_consecutive(room, *args):
    room_slot_pairs = [(args[i], args[i + 1]) for i in range(0, len(args), 2)]
    slots_in_room = sorted(slot for r, slot in room_slot_pairs if r == room and slot is not None)
    for i in range(len(slots_in_room) - 1):
        if slots_in_room[i + 1] - slots_in_room[i] == 1:
            return False
    return True

def get_day_from_slot(slot):
    return (slot - 1) // 4

def max_three_lessons_per_day(*slots):
    slots = [s for s in slots if s is not None] 
    days = [get_day_from_slot(s) for s in slots]
    c = Counter(days)
    return all(v <= 3 for v in c.values())


# =============================
# 1. Importar datasets
# =============================
dataset_path = "datasets/timetable_dataset.txt"

courses_assigned_to_classes = {}
courses_assigned_to_lecturers = {}
timeslot_restrictions = {}
roomrestrictions = {}
online_classes = {}
all_courses_with_variables = set() 

try:
    with open(dataset_path, "r", encoding="utf-8-sig") as f:
        lines = f.readlines()

    reading_section = None
    for line in lines:
        line = line.strip()
        if not line:
            continue

        elif line.startswith("#cc"): reading_section = "cc"
        elif line.startswith("#dsd"): reading_section = "dsd"
        elif line.startswith("#tr"): reading_section = "tr"
        elif line.startswith("#rr"): reading_section = "rr"
        elif line.startswith("#oc"): reading_section = "oc"
        elif line.startswith("#"): reading_section = None
        else:
            if reading_section == "cc":
                parts = line.split()
                courses_assigned_to_classes[parts[0]] = parts[1:]
                for course in parts[1:]:
                     all_courses_with_variables.add(course)
            elif reading_section == "dsd":
                parts = line.split()
                courses_assigned_to_lecturers[parts[0]] = parts[1:]
            elif reading_section == "tr":
                parts = line.split()
                timeslot_restrictions[parts[0]] = list(map(int, parts[1:]))
            elif reading_section == "rr":
                parts = line.split()
                roomrestrictions[parts[0]] = parts[1:]
            elif reading_section == "oc":
                parts = line.split()
                online_classes[parts[0]] = list(map(int, parts[1:]))
                
except FileNotFoundError:
    print(f"ERRO: Ficheiro não encontrado no caminho: {dataset_path}")
    print("Usando dados simulados para continuar o exemplo.")
    courses_assigned_to_classes = {'t01': ['UC11', 'UC12', 'UC13', 'UC14', 'UC15', 'UC21', 'UC22', 'UC23', 'UC24', 'UC25', 'UC31', 'UC32', 'UC33', 'UC34', 'UC35']}
    courses_assigned_to_lecturers = {'jo': ['UC11', 'UC21', 'UC22', 'UC31'], 'mike': ['UC12', 'UC23', 'UC32'], 'rob': ['UC13', 'UC14', 'UC24', 'UC33'], 'sue': ['UC15', 'UC25', 'UC34', 'UC35']}
    timeslot_restrictions = {'mike': [13, 14, 15, 16, 17, 18, 19, 20], 'rob': [1, 2, 3, 4], 'sue': [9, 10, 11, 12, 17, 18, 19, 20]}
    roomrestrictions = {'UC14': ['Lab01'], 'UC22': ['Lab01']}
    online_classes = {'UC21': [2], 'UC31': [2]}
    all_courses_with_variables = set(courses_assigned_to_classes['t01'])

# =============================
# 2️. Criar CSP e Restrições (O código de restrições foi mantido e limpo)
# =============================

problem = Problem()

# 2.1 - Criar variáveis (Criação de variáveis... omitido para brevidade, mas o código original está correto)
for class_name, courses in courses_assigned_to_classes.items():
    for course in courses:
        for lesson in range(1, lessons_per_course + 1):
            slot_var = f"{course}_{lesson}_slot"
            room_var = f"{course}_{lesson}_room"
            teacher_var = f"{course}_{lesson}_teacher"
            
            problem.addVariable(slot_var, slots)
            
            if course in roomrestrictions:
                problem.addVariable(room_var, roomrestrictions[course])
            else:
                problem.addVariable(room_var, rooms)

            teachers_for_course = [t for t, c_list in courses_assigned_to_lecturers.items() if course in c_list]
            if teachers_for_course:
                problem.addVariable(teacher_var, teachers_for_course)

# 2.2 a 2.8 - Adição de Restrições (O código é o mesmo do post anterior, garantindo a existência de variáveis)


# 2.2 - Nenhum slot consecutivo para a mesma turma (pairwise)
for class_name, courses in courses_assigned_to_classes.items():
    # gerar todas as variáveis de slot dessa turma
    slot_vars = [f"{course}_{lesson}_slot" for course in courses for lesson in range(1, lessons_per_course + 1)]
    # para cada par distinto, proibir slots consecutivos
    for i in range(len(slot_vars)):
        for j in range(i+1, len(slot_vars)):
            a = slot_vars[i]
            b = slot_vars[j]
            if a in problem._variables and b in problem._variables:
                problem.addConstraint(lambda s1, s2: abs(s1 - s2) != 1, (a, b))

# 2.3 - Nenhuma aula consecutiva na mesma sala (pairwise)
# para cada par de aulas distintas: se estiverem na mesma sala, não podem ser em slots consecutivos
all_course_list = list(all_courses_with_variables)
for i in range(len(all_course_list)):
    for j in range(i+1, len(all_course_list)):
        c1 = all_course_list[i]
        c2 = all_course_list[j]
        for l1 in range(1, lessons_per_course + 1):
            for l2 in range(1, lessons_per_course + 1):
                room1 = f"{c1}_{l1}_room"
                slot1 = f"{c1}_{l1}_slot"
                room2 = f"{c2}_{l2}_room"
                slot2 = f"{c2}_{l2}_slot"
                if room1 in problem._variables and room2 in problem._variables and slot1 in problem._variables and slot2 in problem._variables:
                    def no_same_room_consec(r1, s1, r2, s2):
                        # se mesma sala, não podem ser consecutivos
                        if r1 == r2:
                            return abs(s1 - s2) != 1
                        return True
                    problem.addConstraint(no_same_room_consec, (room1, slot1, room2, slot2))

# 2.4 - Disponibilidade dos professores
for teacher, courses in courses_assigned_to_lecturers.items():
    unavailable_slots = timeslot_restrictions.get(teacher, [])
    for course in courses:
        if course in all_courses_with_variables: 
            for lesson in range(1, lessons_per_course + 1):
                slot_var = f"{course}_{lesson}_slot"
                if slot_var in problem._variables:
                    problem.addConstraint(
                        lambda s, unav=unavailable_slots: s not in unav,
                        (slot_var,)
                    )

# 2.5 - Turmas não podem ter duas aulas no mesmo slot
for class_name, courses in courses_assigned_to_classes.items():
    lesson_vars = [f"{course}_{lesson}_slot" for course in courses for lesson in range(1, lessons_per_course + 1)]
    problem.addConstraint(AllDifferentConstraint(), lesson_vars)

# 2.6 - Professores não podem dar duas aulas ao mesmo tempo
for teacher, courses in courses_assigned_to_lecturers.items():
    lesson_vars = []
    for course in courses:
        for lesson in range(1, lessons_per_course + 1):
            slot_var = f"{course}_{lesson}_slot"
            if slot_var in problem._variables:
                lesson_vars.append(slot_var)
    if lesson_vars:
        problem.addConstraint(AllDifferentConstraint(), lesson_vars)


# 2.7 - Aulas online
for course, week_indexes in online_classes.items():
    if course in all_courses_with_variables: 
        for lesson in week_indexes:
            room_var = f"{course}_{lesson}_room"
            if room_var in problem._variables: 
                problem.addConstraint(
                    lambda r: r == "Online",
                    (room_var,)
                )

# 2.8 - Limite máximo de 3 aulas por dia
for class_name, courses in courses_assigned_to_classes.items():
    lesson_vars = [f"{course}_{lesson}_slot" for course in courses for lesson in range(1, lessons_per_course + 1)]
    problem.addConstraint(max_three_lessons_per_day, lesson_vars)


# =============================
# 3️⃣ Resolver e Exportar JSON
# =============================

print("\n--- INÍCIO DA PROCURA ---\n")

start = time.time()
solution = problem.getSolution() 
print("Tempo de execução:", time.time() - start, "segundos")

output_filepath = r"outputs\best_schedule.json"

if solution:
    print(f"✅ Primeira solução encontrada. Exportando para {output_filepath}")
    

    # 3.1 - Preparar os dados para JSON (Estruturar o resultado)
    schedule = {}
    
    for var, value in solution.items():
        parts = var.split('_')
        uc = parts[0]
        lesson_index = parts[1]
        var_type = parts[2]
        
        lesson_key = f"{uc}_{lesson_index}"
        
        if lesson_key not in schedule:
            schedule[lesson_key] = {}
        
        schedule[lesson_key][var_type] = value

    # 3.2 - Escrever no ficheiro JSON
    try:
        with open(output_filepath, 'w', encoding='utf-8') as f:
            json.dump(schedule, f, indent=4)
        print("\nSucesso na exportação.")
        
    except Exception as e:
        print(f"\nERRO ao escrever no arquivo {output_filepath}: {e}")
        print("\n--- Solução JSON (Se falha na escrita) ---")
        print(json.dumps(schedule, indent=4))
        
else:
    print("\n❌ Nenhuma solução encontrada.")
    empty_domains = [v for v, d in problem._variables.items() if not d]
    if empty_domains:
        print("Variáveis com domínio vazio, o que prova a insatisfação:", empty_domains)
    else:
        print("Todas as variáveis têm domínios, mas o problema é demasiado restrito.")
        
if solution:
    print("\n--- Exemplo de Atribuições (30 primeiras) ---")
    for i, var in enumerate(sorted(solution)):
        if i < 200:
            print(f"{var} → {solution[var]}")
        else:
            print("...")
            break



--- INÍCIO DA PROCURA ---

Tempo de execução: 0.005537748336791992 segundos
✅ Primeira solução encontrada. Exportando para outputs\best_schedule.json

Sucesso na exportação.

--- Exemplo de Atribuições (30 primeiras) ---
UC11_1_room → Online
UC11_1_slot → 14
UC11_1_teacher → jo
UC12_1_room → Online
UC12_1_slot → 12
UC12_1_teacher → mike
UC13_1_room → Room2
UC13_1_slot → 20
UC13_1_teacher → rob
UC14_1_room → Lab01
UC14_1_slot → 18
UC14_1_teacher → rob
UC15_1_room → Room2
UC15_1_slot → 16
UC15_1_teacher → sue
UC21_1_room → Online
UC21_1_slot → 17
UC21_1_teacher → jo
UC22_1_room → Lab01
UC22_1_slot → 13
UC22_1_teacher → jo
UC23_1_room → Room2
UC23_1_slot → 11
UC23_1_teacher → mike
UC24_1_room → Online
UC24_1_slot → 19
UC24_1_teacher → rob
UC25_1_room → Room1
UC25_1_slot → 15
UC25_1_teacher → sue
UC31_1_room → Room2
UC31_1_slot → 20
UC31_1_teacher → jo
UC32_1_room → Online
UC32_1_slot → 10
UC32_1_teacher → mike
UC33_1_room → Online
UC33_1_slot → 17
UC33_1_teacher → rob
UC34_1_room → Onlin